In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [3]:
def get_book_and_return_first_line(symb):
    #get ASK and BID price and volume
    #returning BookInfo
    mt5.market_book_add(symb)
    time.sleep(0.1)
    items = mt5.market_book_get(symb)
    mt5.market_book_release(symb)  
    if len(items) > 0:
        for l in range(0,len(items)):
            if l > 0:
                if items[l].type != items[l-1].type:
                    return {'bid': items[l], 'ask': items[l-1]}

# bid - [BookInfo(type=2, price=1.02, volume=41100, volume_dbl=41100.0),
# ask - [BookInfo(type=1, price=1.07, volume=41100, volume_dbl=41100.0)]

In [13]:
   
def butterfly_realtime(tick_data, cost_limit = 1, show_broken_wings = False):
    
    # math = buy 1 -> sell 2 next strike -> buy 1 after next
    # return how much you need to pay to mount this structur
    # if negative you receive money to create it
    # check if butterfly have symmetric wings
    
    model_return = []
    for i in range(0,len(tick_data)-2):
        assymetric_wings = 0
        
        #check if are symmetric wings
        diff_wings = tick_data[i].strike - (2 * tick_data[i + 1].strike) + tick_data[i + 2].strike
        
        #tic = time.perf_counter()
        wing_1 = get_book_and_return_first_line(tick_data[i].option_name)
        wing_2 = get_book_and_return_first_line(tick_data[i+1].option_name)
        wing_3 = get_book_and_return_first_line(tick_data[i+2].option_name)
        #toc = time.perf_counter()
        #print(f"Time to request data {toc - tic:0.4f} seconds")

        if wing_1 != None:
            wing_1_price = wing_1['ask'].price
            wing_1_volume = wing_1['ask'].volume
        else: 
            wing_1_price = tick_data[i].ask
            wing_1_volume = 0
            if float(wing_1_price) == 0:
                continue
                
        if wing_2 != None:
            wing_2_price = wing_2['bid'].price
            wing_2_volume = wing_2['bid'].volume
        else: 
            wing_2_price = tick_data[i+1].bid
            wing_2_volume = 0
            if float(wing_2_price) == 0:
                continue           

        if wing_3 != None:
            wing_3_price = wing_3['ask'].price
            wing_3_volume = wing_3['ask'].volume
            if float(wing_3_price) == 0:
                continue
        else: 
            wing_3_price = tick_data[i+2].ask
            wing_3_volume = 0
            if float(wing_3_price) == 0:
                continue
            
        wing_1_price = round(wing_1_price,2)
        wing_2_price = round(wing_2_price,2)
        wing_3_price = round(wing_3_price,2)
        
        cost =  round(wing_1_price - (2 * wing_2_price) + wing_3_price,2)
        
        max_profit = round((tick_data[i + 1].strike - tick_data[i].strike) - cost,2)
        stock_price = float(tick_data[i].stock_price)
        percent_to_max_profit = round(((tick_data[i + 1].strike - stock_price) / stock_price)*100,2)
        max_loss = cost
        
        if diff_wings > 0 and show_broken_wings == False:
            break
        else:
            max_loss = cost
            
        if cost < cost_limit:
            max_volume = min([wing_1_volume,wing_2_volume,wing_3_volume])
            #'{}:{}:{}'.format('teste','1','ou')
            butterfly_symbols = '{}: {}, {}: {}, {}:{} '.format(tick_data[i].option_name
                                                , wing_1_price
                                                , tick_data[i + 1].option_name
                                                , wing_2_price
                                                , tick_data[i + 2].option_name
                                                , wing_3_price)
            model_return.append(
                {
                 'BUTTERFLY': butterfly_symbols
                 , "COST": cost             
                 , "MAX_PROFIT": max_profit
                 , "MAX_LOSS": max_loss
                 , "MAX_VOLUME" : max_volume                    
                 , "DIFF_BETWEEN_WINGS": diff_wings
                 , "PERCENT_2_MAX_PAYOFF":percent_to_max_profit
                })

    return model_return

    

In [14]:
def butterfly_cost(tick_data, cost_limit = 1, show_broken_wings = False):
    
    # math = buy 1 -> sell 2 next strike -> buy 1 after next
    # return how much you need to pay to mount this structur
    # if negative you receive money to create it
    # check if butterfly have symmetric wings
    
    model_return = []
    for i in range(0,len(tick_data)-2):
        assymetric_wings = 0
        
        #check if are symmetric wings
        diff_wings = tick_data[i].strike - (2 * tick_data[i + 1].strike) + tick_data[i + 2].strike
        
        wing_1_price = tick_data[i].ask
        wing_2_price = tick_data[i + 1].bid
        wing_3_price = tick_data[i + 2].ask
        if float(wing_1_price) == 0 or float(wing_2_price) == 0 or float(wing_3_price) == 0:
            continue
        
        cost =  wing_1_price - (2 * wing_2_price) + wing_3_price
         
        max_profit = round((tick_data[i + 1].strike - tick_data[i].strike) - cost,2)
        stock_price = float(tick_data[i].stock_price)
        percent_to_max_profit = round(((tick_data[i + 1].strike - stock_price) / stock_price)*100,2)
        max_loss = cost
        
        if diff_wings > 0 and show_broken_wings == False:
            continue
        else:
            max_loss = cost
            
        if cost < cost_limit:
            butterfly_symbols = '{}: {}, {}: {}, {}:{} '.format(tick_data[i].option_name
                                                , round(wing_1_price,2)
                                                , tick_data[i + 1].option_name
                                                , round(wing_2_price,2)
                                                , tick_data[i + 2].option_name
                                                , round(wing_3_price,2))            
            model_return.append(
                {
                 'BUTTERFLY': butterfly_symbols
                 , "COST": cost             
                 , "MAX_PROFIT": max_profit
                 , "MAX_LOSS": max_loss                 
                 , "DIFF_BETWEEN_WINGS": diff_wings
                 , "PERCENT_2_MAX_PAYOFF":percent_to_max_profit
                })

    return model_return
 

In [15]:
updated_ticks = estrategies.update_quotes(just_last_update=True,mode=InformationType.Offline)
       

In [16]:
this_due = updated_ticks[0]
next_due = updated_ticks[1]
after_next_due = updated_ticks[1]


In [27]:
butterfly_offline_data_dict = butterfly_realtime(this_due,0.03,True)
butterfly_offline_data_dict

[]